<a href="https://colab.research.google.com/github/dwahast/Deep-learning/blob/master/MLP_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons
- Link útil (https://towardsdatascience.com/cifar-10-image-classification-in-tensorflow-5b501f7dc77c)

In [0]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

import torch.nn.functional as F # softmax

#import pandas as pd # organize files 


In [2]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

0it [00:00, ?it/s]

100%|█████████▉| 169713664/170498071 [00:27<00:00, 7613605.81it/s]

Files already downloaded and verified


In [3]:
train_loader = DataLoader(dataset=dataset_train, shuffle=True)
test_loader = DataLoader(dataset=dataset_test, shuffle=False)
print(len(test_loader))

10000


In [0]:
# Definir a arquitetura MLP

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32*32, 20)
        self.fc2 = nn.Linear(20, 10)
        self.activation_function = nn.Sigmoid()
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        return x
# Arquitecture with RELU      
class MLP_relu(nn.Module):
    def __init__(self):
        super(MLP_relu, self).__init__()
        self.fc1 = nn.Linear(32*32, 20)
        self.fc2 = nn.Linear(20, 10)
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x),dim =1)
        return x
      
# New Arquitecture with RELU   
class MLP_Srelu(nn.Module):
    def __init__(self):
        super(MLP_Srelu, self).__init__()
        self.fc1 = nn.Linear(32*32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 10)
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x),dim =1)
        return x   


In [0]:
model_std = MLP()
model_relu = MLP_relu()
model_Srelu = MLP_Srelu()
#print(model)

In [0]:
# Definir otimizador e loss
# Nota: testar outros otimizadores e funções de loss (em particular cross entropy)

def optimizer_set(model,optimizer_method, learning_rate):
  if optimizer_method == "sgd":
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
  elif optimizer_method == "adadelta":
    optimizer = torch.optim.Adadelta(model.parameters(), lr = learning_rate)
  elif optimizer_method == "adagrad":
    optimizer = torch.optim.Adagrad(model.parameters(), lr = learning_rate)
  elif optimizer_method == "adamax":
    optimizer = torch.optim.Adamax(model.parameters(), lr = learning_rate)
  elif optimizer_method == "rms":
    optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate)
  return optimizer

def loss_function_set(loss_function):
  if loss_function == "mse":
    loss_fn = torch.nn.MSELoss()
  elif loss_function == "nllos":
     loss_fn = torch.nn.NLLoss()
  elif loss_function == "cross":
     loss_fn = torch.nn.CrossEntropyLoss()
  return loss_fn
  

In [0]:
def one_hot(label, output_size):
    
    label_select = np.zeros(output_size)
    label_select[label]=1
    
    return torch.Tensor(label_select)

In [0]:
def evaluate(model, loss_function, dataset):
  model.eval() # set model to Evaluate "mode"
 # losses = []
  corrects = 0
  current_total = 0
  accuracies = 0
  
 # loss_fn = loss_function_set(loss_function)
  
  for image, label in dataset:
    outputs = model(image)
    
   # if(loss_function == 'mse'):
   #   label_target = one_hot(label,10) # one_hot to set the training class in the escalar
   #   loss = loss_fn(outputs,label_target)
   # else:
   #   loss = loss_fn(outputs,label)
      
   # losses.append(loss.item())
    
    _, predicted = torch.max(outputs.data, 1)
    
    if(predicted==label):
      corrects += 1
    
    current_total += 1
  
  accuracy = corrects/current_total

  return accuracy #, np.mean(losses)
      

#Training function

In [0]:
# Realizar o treinamento aqui
def fit(model, epochs, optimizer_method, loss_function, learning_rate):
  
  print("Opt Method:", optimizer_method.upper(), "| Loss Function:", loss_function.upper(), "| Learning Rate:", learning_rate)
  optimizer = optimizer_set(model,optimizer_method, learning_rate) #optimizer and Learning rate setter
  loss_fn = loss_function_set(loss_function) 
  accuracies = []
  losses = []
  train_acc = []
  for epoch in range(epochs):
    model.train() # Set model to TRAIN "mode" (can be set to False for Test)
    epoch_losses = []
    for image,label in train_loader:
      optimizer.zero_grad()  # cleaning gradients between mini batches
      outputs = model(image) 
     
      if(loss_function == 'mse'):
        label_target = one_hot(label,10) # one_hot to set the training class in the escalar
        loss = loss_fn(outputs,label_target)
      else: 
        loss = loss_fn(outputs, label)
        
      loss.backward() # Backpropagation 
      optimizer.step() # Optimization Method
      
      epoch_losses.append(loss.item())

    losses.append(np.mean(epoch_losses)) # Append mean losses for each epoch
   
    
    acc = evaluate(model, loss_function, train_loader)
    train_acc.append(acc)
    acc = evaluate(model, loss_function, test_loader) #evaluate
    accuracies.append(acc)
    
    #if(epoch%10==0):
    print("Epoch:",epoch, "- Average loss:", np.mean(epoch_losses),"- Accuracy:", acc) # Print mean Loss for each epoch
 
  return {
      "Model": outputs,
      "Acc": accuracies,
      "Train Acc": train_acc,
      "Loss": losses,
      "Name": optimizer_method + " | " + loss_function + " | " + str(learning_rate),
      "Arc": model
      #"Evaluate Train Loss": train_losses,
      
  }

# Training Your Model
- Variable "trained_models" append all models trained
- function "fit" train and return the Model and the all variable that will be used latter
  - *fit(Model, Epochs, Optimizer method, Loss function, Learning rate, momentum)*

In [0]:
# Avaliar o modelo aqui (no conjunto de teste)
trained_models = []
ep = 15 #epocas
trained_models.append(fit(model_Srelu,ep,"sgd", "cross", 0.01))


In [0]:
trained_models.append(fit(model_relu,ep,"sgd",   "cross", 0.01))

In [0]:
trained_models.append(fit(model_std,ep,"sgd", "cross", 0.01))

# Text and Graphical Output

In [0]:
plt.title("\nAccuracy", fontsize=20, loc="left")
for models in trained_models:
  plt.plot(models["Acc"], label=models["Name"])
  plt.plot(models["Train Acc"],"--") 
  print(models["Arc"],"\n",models["Name"], "| ACC: ", np.mean(models["Acc"]),"\n")

plt.legend(loc='best')
plt.show